<a href="https://colab.research.google.com/github/GoldenScientist/Big-Data-Analysis-with-Scala-and-Spark/blob/main/BORALEX_INTERVIEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

L’énergie renouvelable provient de sources renouvelées en permanence par la nature. Il existe deux catégories principales : du Soleil/Lune (cycles de l’eau, mouvements de la mer, vent et plantes en croissance) et de la Terre (sources de chaleur). Nous entendons souvent parler de transition énergétique, et les sources d’énergie renouvelables se développent de manière impressionnante autour de nous. Mais ces énergies fournissent-elles une part importante de la production ? Les ressources naturelles sont-elles utilisées efficacement? Nous avons essayé d’explorer la production et la consommation d’énergie éolienne et solaire en France pour 2014 - 2020 afin de répondre à ces questions. 
Nous avons utilisé les données d’Open Data Réseaux Energies :



*   https://odre.opendatasoft.com/explore/dataset/rayonnement-solaire-vitesse-vent-tri-horaires-regionaux/export/?disjunctive.region
*   https://odre.opendatasoft.com/explore/dataset/parc-regional-annuel-prod-eolien-solaire/export/?disjunctive.region


*   https://odre.opendatasoft.com/explore/dataset/equilibre-regional-mensuel-prod-conso-brute/export/?disjunctive.region&sort=mois
*  https://odre.opendatasoft.com/explore/dataset/parc-regional-annuel-prod-eolien-solaire/export/?disjunctive.region&sort=annee&location=5,46.44448,2.22123&basemap=jawg.light
*  https://odre.opendatasoft.com/explore/dataset/fc-tc-regionaux-mensuels-enr/information/?disjunctive.region
* https://odre.opendatasoft.com/explore/dataset/rayonnement-solaire-vitesse-vent-tri-horaires-regionaux/information/?disjunctive.region







In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!jupyter nbconvert --to html /content/BORALEX_INTERVIEW.ipynb

[NbConvertApp] WARNING | pattern '/content/BORALEX_INTERVIEW.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True

In [ ]:
# installations pour résoudre les problèmes de dépendances
!pip install typing-extensions
!pip install wheel
# installation des librairies prefect dont le role est de suivre, coordonner et orchestrer les flux de données entre les applications
!pip install h5py
! pip install -U prefect
!pip install prefect-dask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from prefect import flow, task
import httpx
from prefect_dask import DaskTaskRunner



In [ ]:
# chargement des fichiers en local en utilisant la librairie google colab
from google.colab import files
data_to_load = files.upload()

Saving rayonnement-solaire-vitesse-vent-tri-horaires-regionaux.csv to rayonnement-solaire-vitesse-vent-tri-horaires-regionaux.csv
Saving fc-tc-regionaux-mensuels-enr.csv to fc-tc-regionaux-mensuels-enr.csv
Saving equilibre-regional-mensuel-prod-conso-brute.csv to equilibre-regional-mensuel-prod-conso-brute.csv
Saving production-regionale-mensuelle-filiere.csv to production-regionale-mensuelle-filiere.csv
Saving parc-regional-annuel-prod-eolien-solaire.csv to parc-regional-annuel-prod-eolien-solaire.csv


Vue d'ensemble des sources d'énergies en France

In [ ]:
#  lecture du fichier roduction-regionale-mensuelle-filiere.csv 
prod_region = pd.read_csv("production-regionale-mensuelle-filiere.csv",sep=';',engine='python',quotechar='"',encoding='utf-8')

In [ ]:
prod_region

,Mois,Code INSEE région,Région,Production nucléaire (GWh),Production thermique (GWh),Production hydraulique (GWh),Production éolienne (GWh),Production solaire (GWh),Production bioénergies (GWh),Géo-shape région,Géo-point région
0,2014-01,28,Normandie,7354.715338,402.534901,16.013500,156.651038,2.868850,39.911126,"{""coordinates"": [[[0.7978593666, 48.1947692642...","49.120061533,0.11093829100000001"
1,2014-01,32,Hauts-de-France,4008.641360,502.547223,1.564361,525.159000,2.916058,70.185189,"{""coordinates"": [[[4.2331585664, 49.95775098],...","49.969517433,2.771371932"
2,2014-01,52,Pays de la Loire,NaN,377.532172,2.623810,144.194423,9.607937,29.175033,"{""coordinates"": [[[[-2.337432985, 46.719095681...","47.478103923,-0.8167881810000001"
3,2014-01,76,Occitanie,1795.000833,46.772218,1384.716075,189.597415,40.344570,61.104208,"{""coordinates"": [[[[-0.0751953504, 43.30715512...","43.702500286,2.1439233890000002"
4,2014-02,28,Normandie,5173.583790,407.647156,15.164572,166.273040,5.522769,39.145851,"{""coordinates"": [[[0.7978593666, 48.1947692642...","49.120061533,0.11093829100000001"
...,...,...,...,...,...,...,...,...,...,...,...
1243,2021-09,44,Grand Est,4732.839371,479.000278,644.303770,307.103144,106.276879,104.140124,"{""coordinates"": [[[3.4147355035, 48.3901877653...","48.688976812,5.613113265"
1244,2021-12,44,Grand Est,5407.179145,1227.234397,677.181941,889.979217,20.060003,112.168220,"{""coordinates"": [[[3.4147355035, 48.3901877653...","48.688976812,5.613113265"
1245,2021-04,27,Bourgogne-Franche-Comté,NaN,41.642873,41.357840,167.269925,46.366484,33.051392,"{""coordinates"": [[[3.4147355035, 48.3901877653...","47.234282107,4.806856823"
1246,2021-05,27,Bourgogne-Franche-Comté,NaN,28.061116,100.375546,226.399623,43.493538,33.066362,"{""coordinates"": [[[3.4147355035, 48.3901877653...","47.234282107,4.806856823"


In [ ]:
# sauvegarde de lignes et de colonnes de mon DataFrame comme tuple (Ligne, colonne)
prod_region.shape 

(1248, 11)

In [ ]:
prod_region.dtypes.value_counts()

float64    6
object     4
int64      1
dtype: int64

In [ ]:
# detection des valeurs nulles sur les colonnes de mon dataFrame via la fonction isna
prod_region.isna().sum()

Mois                              0
Code INSEE région                 0
Région                            0
Production nucléaire (GWh)      504
Production thermique (GWh)        0
Production hydraulique (GWh)      0
Production éolienne (GWh)         0
Production solaire (GWh)          0
Production bioénergies (GWh)      0
Géo-shape région                  0
Géo-point région                  0
dtype: int64

In [ ]:
print(f'{prod_region["Production nucléaire (GWh)"].isna().sum()/len(prod_region):.1%} missing values') 

40.4% missing values


In [ ]:
# Ajout du Total et groupons par region
prod_region_treated = pd.concat([prod_region,pd.Series(prod_region.iloc[:,3:9].sum(axis=1),name='Total (GWh)')],axis=1)
prod_region_treated.drop(columns=["Géo-shape région","Géo-point région"]).groupby(["Région","Mois"]).sum()

Code INSEE région  Production nucléaire (GWh)  \
Région               Mois                                                     
Auvergne-Rhône-Alpes 2014-01                 84                 9578.044203   
                     2014-02                 84                 8105.587206   
                     2014-03                 84                 7702.312262   
                     2014-04                 84                 6649.736215   
                     2014-05                 84                 6671.538747   
...                                         ...                         ...   
Île-de-France        2021-08                 11                    0.000000   
                     2021-09                 11                    0.000000   
                     2021-10                 11                    0.000000   
                     2021-11                 11                    0.000000   
                     2021-12                 11                    0.000000   

                              Production thermique (GWh)  \
Région               Mois                                  
Auvergne-Rhône-Alpes 2014-01                  139.691811   
                     2014-02                  125.700315   
                     2014-03                  139.208746   
                     2014-04                   25.317769   
                     2014-05                   21.833313   
...                                                  ...   
Île-de-France        2021-08                    3.653131   
                     2021-09                    1.040949   
                     2021-10                   33.053277   
                     2021-11                  412.144446   
                     2021-12                  425.024035   

                              Production hydraulique (GWh)  \
Région               Mois                                    
Auvergne-Rhône-Alpes 2014-01                   3116.187950   
                     2014-02                   2916.083533   
                     2014-03                   2781.287120   
                     2014-04                   2287.738179   
                     2014-05                   2525.021294   
...                                                    ...   
Île-de-France        2021-08                      6.373146   
                     2021-09                      4.661814   
                     2021-10                      5.355077   
                     2021-11                      5.509420   
                     2021-12                      4.724080   

                              Production éolienne (GWh)  \
Région               Mois                                 
Auvergne-Rhône-Alpes 2014-01                  75.012542   
                     2014-02                  80.169361   
                     2014-03                  70.370852   
                     2014-04                  57.750445   
                     2014-05                  89.805539   
...                                                 ...   
Île-de-France        2021-08                  19.082232   
                     2021-09                  14.104310   
                     2021-10                  28.332040   
                     2021-11                  23.065084   
                     2021-12                  34.068912   

                              Production solaire (GWh)  \
Région               Mois                                
Auvergne-Rhône-Alpes 2014-01                 18.500091   
                     2014-02                 26.686280   
                     2014-03                 59.531770   
                     2014-04                 65.104183   
                     2014-05                 73.097286   
...                                                ...   
Île-de-France        2021-08                 20.073196   
                     2021-09                 18.555532   
                     2021-10                 14.586024   
        

In [ ]:
# Supprimer les non-renouvelables (pas de détails sur l’origine de « thermique », nous le considérerons comme non renouvelable)
prod_region_renouv = prod_region_treated.drop(columns=['Production nucléaire (GWh)','Production thermique (GWh)'])
#  Ajoutons la somme des Energies renouvelables et les totaux séparés
prod_region_renouv = pd.concat([prod_region_renouv,pd.Series(prod_region_renouv.iloc[:,3:6].sum(axis=1),name='Total renouv. (GWh)')],axis=1)
prod_region_renouv["Total non-renouv. (GWh)"] = prod_region_renouv["Total (GWh)"] - prod_region_renouv["Total renouv. (GWh)"]
# Énergies renouvelables regroupées par région
prod_region_renouv.drop(columns=["Géo-shape région","Géo-point région"]).groupby(["Région","Mois"]).sum()

Code INSEE région  Production hydraulique (GWh)  \
Région               Mois                                                       
Auvergne-Rhône-Alpes 2014-01                 84                   3116.187950   
                     2014-02                 84                   2916.083533   
                     2014-03                 84                   2781.287120   
                     2014-04                 84                   2287.738179   
                     2014-05                 84                   2525.021294   
...                                         ...                           ...   
Île-de-France        2021-08                 11                      6.373146   
                     2021-09                 11                      4.661814   
                     2021-10                 11                      5.355077   
                     2021-11                 11                      5.509420   
                     2021-12                 11                      4.724080   

                              Production éolienne (GWh)  \
Région               Mois                                 
Auvergne-Rhône-Alpes 2014-01                  75.012542   
                     2014-02                  80.169361   
                     2014-03                  70.370852   
                     2014-04                  57.750445   
                     2014-05                  89.805539   
...                                                 ...   
Île-de-France        2021-08                  19.082232   
                     2021-09                  14.104310   
                     2021-10                  28.332040   
                     2021-11                  23.065084   
                     2021-12                  34.068912   

                              Production solaire (GWh)  \
Région               Mois                                
Auvergne-Rhône-Alpes 2014-01                 18.500091   
                     2014-02                 26.686280   
                     2014-03                 59.531770   
                     2014-04                 65.104183   
                     2014-05                 73.097286   
...                                                ...   
Île-de-France        2021-08                 20.073196   
                     2021-09                 18.555532   
                     2021-10                 14.586024   
                     2021-11                  7.509567   
                     2021-12                  4.795222   

                              Production bioénergies (GWh)   Total (GWh)  \
Région               Mois                                                  
Auvergne-Rhône-Alpes 2014-01                     74.183339  13001.619936   
                     2014-02                     69.171701  11323.398396   
                     2014-03                     75.699792  10828.410542   
                     2014-04                     57.832202   9143.478993   
                     2014-05                     66.961250   9448.257429   
...                                                    ...           ...   
Île-de-France        2021-08                     96.089244    145.270949   
                     2021-09                    129.321770    167.684375   
                     2021-10                    115.520016    196.846434   
                     2021-11                    105.657454    553.885971   
                     2021-12                     94.803013    563.415262   

                              Total renouv. (GWh)  Total non-renouv. (GWh)  
Région               Mois                                                   
Auvergne-Rhône-Alpes 2014-01          3209.700583              9791.919353  
                     2014-02          3022.939174              8300.459222  
                     2014-03          2911.189742              7917.220800  
                     2014-04          2410.592807              6732.886186  
  

In [ ]:
#verifions si toutes les régions sont incluses
prod_region_renouv["Région"].nunique(),prod_region_renouv["Région"].unique() 

(13, array(['Normandie', 'Hauts-de-France', 'Pays de la Loire', 'Occitanie',
        'Bretagne', 'Bourgogne-Franche-Comté', 'Grand Est',
        'Île-de-France', "Provence-Alpes-Côte d'Azur",
        'Centre-Val de Loire', 'Corse', 'Nouvelle-Aquitaine',
        'Auvergne-Rhône-Alpes'], dtype=object))

In [ ]:
# Graphe récapitulant toutes les productions
fig = px.line(prod_region.drop(columns=["Code INSEE région","Géo-shape région","Géo-point région"]).groupby(["Mois"]).sum(),
              title = 'Evolution of production',
              labels = {"value": "Production, GWh", 
                        "variable": ""},
              height = 500, width = 1000)

fig.show()

In [ ]:
# graphe affichant l'évolution de la production des énergies renouvelables et non renouvelables
fig = px.scatter(prod_region_renouv.filter(items = ["Mois","Total renouv. (GWh)", "Total non-renouv. (GWh)"]).groupby(["Mois"]).sum(),
              #trendline="ols",
              title = 'Evolution of production',
              labels = {"value": "Production, GWh", 
                        "variable": ""}, 
              height = 500, width = 1000)
fig.data[0].update(mode='lines')
fig.data[1].update(mode='lines')

fig.show()

In [ ]:
# Evolutions des énergies renouvelables
perc20 = prod_region_renouv.loc[prod_region_renouv["Mois"] == '2020-12']["Total renouv. (GWh)"].sum() / prod_region_renouv.loc[prod_region_renouv["Mois"] == '2020-12']["Total non-renouv. (GWh)"].sum()
perc14 = prod_region_renouv.loc[prod_region_renouv["Mois"] == '2014-01']["Total renouv. (GWh)"].sum() / prod_region_renouv.loc[prod_region_renouv["Mois"] == '2014-01']["Total non-renouv. (GWh)"].sum()
print(f'Portion of these energy comes from renewable sources (wind, solar, hydro, biomass) in total production increases by {perc20 - perc14:.1%} for given period: from {perc14:.1%} at 2014 to {perc20:.1%} at the end of 2020.')

Portion of these energy comes from renewable sources (wind, solar, hydro, biomass) in total production increases by 5.8% for given period: from 20.3% at 2014 to 26.1% at the end of 2020.


In [ ]:
# Evolution du pourcentage éolien
perc_eol_14 = prod_region_treated.loc[prod_region_treated["Mois"] == '2014-01']["Production éolienne (GWh)"].sum() / prod_region_treated.loc[prod_region_treated["Mois"] == '2014-01']["Total (GWh)"].sum()
perc_eol_20 = prod_region_treated.loc[prod_region_treated["Mois"] == '2020-12']["Production éolienne (GWh)"].sum() / prod_region_treated.loc[prod_region_treated["Mois"] == '2020-12']["Total (GWh)"].sum()
print(f'Portion of wind energy in total production: {perc_eol_14:.1%} to 2014 and {perc_eol_20:.1%} at 2020.')

Portion of wind energy in total production: 3.9% to 2014 and 8.6% at 2020.


In [ ]:
# Evolution du pourcentage solaire
perc_sol_14 = prod_region_treated.loc[prod_region_treated["Mois"] == '2014-01']["Production solaire (GWh)"].sum() / prod_region_treated.loc[prod_region_treated["Mois"] == '2014-01']["Total (GWh)"].sum()
perc_sol_20 = prod_region_treated.loc[prod_region_treated["Mois"] == '2020-12']["Production solaire (GWh)"].sum() / prod_region_treated.loc[prod_region_treated["Mois"] == '2020-12']["Total (GWh)"].sum()
print(f'Portion of solar energy in total production: {perc_sol_14:.1%} to 2014 and {perc_sol_20:.1%} at 2020.')

Portion of solar energy in total production: 0.3% to 2014 and 0.7% at 2020.


In [ ]:
# production par région des énergies éoliennes et solaire
prod_region_renouv["Total-E-S (GWh)"] = prod_region_renouv["Total (GWh)"] - prod_region_renouv["Production éolienne (GWh)"] - prod_region_renouv["Production solaire (GWh)"]

data = prod_region_renouv.drop(columns=["Code INSEE région","Production bioénergies (GWh)"]).sort_values(["Mois","Total-E-S (GWh)"], ascending=[True, True])
fig = px.bar(data, 
            x = ["Production éolienne (GWh)","Production solaire (GWh)","Total-E-S (GWh)"],
            y = "Région",  
            animation_frame = "Mois",
            orientation = 'h',
            title = 'Production par région',
            labels = {"value": "Production, GWh", 
                      "variable": ""})
fig.show()

Nous analyserons les centrales éoliennes et solaires

In [ ]:
# lecture des fichier csv parc-regional-annuel-prod-eolien-solaire
parc_region = pd.read_csv("parc-regional-annuel-prod-eolien-solaire.csv",sep=';',engine='python',quotechar='"',encoding='utf-8')

In [ ]:
# detection des valeurs nulles
parc_region.isna().sum()

Année                          0
Code INSEE région              0
Région                         0
Parc installé éolien (MW)     24
Parc installé solaire (MW)    69
Géo-shape région               0
Géo-point région               0
dtype: int64

In [ ]:
# graphe decrivant l'évolution des installation solaire et éolien
pvt = pd.pivot_table(parc_region, index = "Année", values = ["Parc installé éolien (MW)","Parc installé solaire (MW)"], aggfunc='sum')
fig = px.line(pvt,
              labels = {"value": "MW", 
                        "variable": ""},
              title = 'Evolution of installations')
fig.update_yaxes(range=[0, 20000])

fig.show()

In [ ]:
# graphe regroupant les parcs éoliens et solaires par région
parc_region["Région"] = parc_region["Région"].replace("Île-de-France", "Ile-de-France")
data = parc_region.sort_values(["Année","Région"], ascending=[True,False])
fig = px.bar(
            data, 
            x = ["Parc installé éolien (MW)","Parc installé solaire (MW)"],
            range_x = [0,5500],
            text = "value",
            y = "Région",  
            animation_frame = "Année",
            labels = {"value": "MW", 
                      "variable": ""},
            title = 'Parc des installations éolien et solaire (MW)')
fig.show()

In [ ]:
# Graphe affichant la production moyenne des éolienne et solaire par région
from plotly.subplots import make_subplots

for_plot = pd.pivot_table(prod_region_renouv, index = "Région", values = ["Production éolienne (GWh)","Production solaire (GWh)"], aggfunc='sum')
labels = for_plot.index

fig = make_subplots(rows = 1, cols = 2, specs = [[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels = labels,
                     values = for_plot["Production éolienne (GWh)"],
                     pull = [0,0,0,0,0,0,0.1,0,0,0,0,0,0]), 1,1)
fig.add_trace(go.Pie(labels = labels, 
                     values = for_plot["Production solaire (GWh)"], 
                     pull = [0,0,0,0,0,0,0,0,0.1,0,0,0,0]), 1,2)

fig.update_layout(title_text = "Production moyenne éolienne et solaire par région",
                 annotations=[dict(text='éolienne', x=0.1, y=0.95),
                              dict(text='solaire', x=0.7, y=0.95)])
fig.update_traces(hoverinfo = 'label+percent', textinfo = 'percent')
fig.show()

In [ ]:
print(f'Top 5 production éolienne: {prod_region_renouv.groupby(["Région"]).sum().sort_values(["Production éolienne (GWh)"], ascending=False).head().index}')

Top 5 production éolienne: Index(['Hauts-de-France', 'Grand Est', 'Occitanie', 'Centre-Val de Loire',
       'Bretagne'],
      dtype='object', name='Région')


In [ ]:
print(f'Top 5 production solaire: {prod_region_renouv.groupby(["Région"]).sum().sort_values(["Production solaire (GWh)"], ascending=False).head().index}')

Top 5 production solaire: Index(['Nouvelle-Aquitaine', 'Occitanie', 'Provence-Alpes-Côte d'Azur',
       'Auvergne-Rhône-Alpes', 'Grand Est'],
      dtype='object', name='Région')


In [ ]:
df = pd.pivot_table(prod_region_renouv, index = "Mois", columns = "Région", values = "Production éolienne (GWh)", aggfunc='sum')

fig = go.Figure()
fig.add_trace(go.Scatter(x = df.index, y = df['Hauts-de-France'].values, name = 'Hauts-de-France'))
fig.add_trace(go.Scatter(x = df.index, y = df['Grand Est'].values, name = 'Grand Est'))
fig.add_trace(go.Scatter(x = df.index, y = df['Occitanie'].values, name = 'Occitanie'))
fig.update_layout(title_text = "Top 3 production éolienne")
fig.show()

In [ ]:
df = pd.pivot_table(prod_region_renouv, index = "Mois", columns = "Région", values = "Production solaire (GWh)", aggfunc='sum')

fig = go.Figure()
fig.add_trace(go.Scatter(x = df.index, y = df['Nouvelle-Aquitaine'].values, name = 'Nouvelle-Aquitaine'))
fig.add_trace(go.Scatter(x = df.index, y = df['Occitanie'].values, name = 'Occitanie'))
fig.add_trace(go.Scatter(x = df.index, y = df["Provence-Alpes-Côte d'Azur"].values, name = "Provence-Alpes-Côte d'Azur"))
fig.update_layout(title_text = "Top 3 production solaire")
fig.show()

Pour les deux sources, la production semble cyclique. Cela peut être lié aux demandes saisonnières, mais comme cette portion d’énergie est relativement petite, nous pouvons également nous attendre à ce que leur dépendance aux conditions météorologiques saisonnières.Tout d’abord, nous pouvons comparer ces conditions avec l’efficacité des installations par le paramètre FC (« facteur de charge ») décrivant l’énergie générée liée à l’énergie maximale possible que cette station pourrait produire si elle fonctionnait sur 100% de sa capacité.


In [ ]:
ray_vit = pd.read_csv("rayonnement-solaire-vitesse-vent-tri-horaires-regionaux.csv",sep=';',engine='python',quotechar='"',encoding='utf-8')

In [ ]:
ray_vit["Date"] = [x[:7] for x in ray_vit["Date"]]
ray_vit_treated = pd.pivot_table(ray_vit, index = ["Date","Région"], values = ["Vitesse du vent à 100m (m/s)","Rayonnement solaire global (W/m2)"], aggfunc='mean').reset_index().rename(columns={"Date": "Mois"})
ray_vit_treated

,Mois,Région,Rayonnement solaire global (W/m2),Vitesse du vent à 100m (m/s)
0,2016-01,Auvergne-Rhône-Alpes,54.626559,2.973482
1,2016-01,Bourgogne-Franche-Comté,44.291700,5.730607
2,2016-01,Bretagne,40.703725,10.414089
3,2016-01,Centre-Val de Loire,40.527611,7.466964
4,2016-01,Corse,76.961822,5.793239
...,...,...,...,...
1035,2022-08,Nouvelle-Aquitaine,6.650000,3.770000
1036,2022-08,Occitanie,4.400000,5.120000
1037,2022-08,Pays de la Loire,8.520000,4.710000
1038,2022-08,Provence-Alpes-Côte d'Azur,2.560000,4.610000


In [ ]:
fc_tc = pd.read_csv("fc-tc-regionaux-mensuels-enr.csv",sep=';',engine='python',quotechar='"',encoding='utf-8')

In [ ]:
# grouped
fig = px.line(fc_tc.filter(items = ["Mois", "FC moyen mensuel éolien", "FC moyen mensuel solaire"]).groupby(["Mois"]).mean(),
              title = 'FC solaire/éolien',
              labels = {"value": "%", 
                        "variable": ""},)
fig.show()

In [ ]:
fc_tc["FC moyen mensuel éolien"].max(), fc_tc["FC moyen mensuel solaire"].max()

(60.52, 33.18)

In [ ]:
fc_tc["FC moyen mensuel éolien"].mean(), fc_tc["FC moyen mensuel solaire"].mean()

(22.580512820512823, 13.684222756410255)

Maximun de pourcentages enregistrés pour chaque source d'énergie 

In [ ]:
# Max éolien ou et quand
fc_tc.loc[fc_tc["FC moyen mensuel éolien"].idxmax()]

Mois                                                                      2020-02
Code INSEE région                                                              11
Région                                                              Île-de-France
FC moyen mensuel hydraulique                                                12.45
FC moyen mensuel éolien                                                     60.52
FC moyen mensuel solaire                                                     6.51
TC moyen mensuel hydraulique                                                 0.03
TC moyen mensuel éolien                                                      0.59
TC moyen mensuel solaire                                                     0.09
TC moyen mensuel bioénergies                                                 0.83
Géo-shape région                {"coordinates": [[[3.4850192327, 48.8518453231...
Géo-point région                                           48.7093154,2.503402421
Name: 296, dtype

In [ ]:
# Max solaire ou et quand
fc_tc.loc[fc_tc["FC moyen mensuel solaire"].idxmax()]

Mois                                                                      2018-07
Code INSEE région                                                              44
Région                                                                  Grand Est
FC moyen mensuel hydraulique                                                28.78
FC moyen mensuel éolien                                                     10.08
FC moyen mensuel solaire                                                    33.18
TC moyen mensuel hydraulique                                                14.54
TC moyen mensuel éolien                                                      7.02
TC moyen mensuel solaire                                                     3.51
TC moyen mensuel bioénergies                                                 2.06
Géo-shape région                {"coordinates": [[[3.4147355035, 48.3901877653...
Géo-point région                                         48.688976812,5.613113265
Name: 536, dtype

cependant les deux sources n’ont jamais été utilisées à pleine capacité, leurs profils sont cycliques et opposés ce qui nous oblige à vérifier les conditions météorologiques.



In [ ]:
#fc + soleil
subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.line(fc_tc.filter(items = ["Mois", "Région", "FC moyen mensuel solaire"]).groupby(["Mois"]).mean(),
              labels = {"value": "%", "variable": ""})
fig2 = px.line(ray_vit_treated.drop(columns = ["Vitesse du vent à 100m (m/s)"]).groupby(["Mois"]).mean(), 
               labels = {"value": "W/m2", "variable": ""},
               color_discrete_map = {"Rayonnement solaire global (W/m2)": "red"})

fig2.update_traces(yaxis = "y2", line = dict(width = 2,dash = 'dot'))

subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title = "Mois"
subfig.layout.yaxis2.title = "Rayonnement solaire global (W/m2)"
subfig.layout.yaxis.title = "FC, %"

subfig.show()

In [ ]:
#fc + vent
subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.line(fc_tc.filter(items = ["Mois", "Région", "FC moyen mensuel éolien"]).groupby(["Mois"]).mean(),
              labels = {"value": "%", "variable": ""})
fig2 = px.line(ray_vit_treated.drop(columns = ["Rayonnement solaire global (W/m2)"]).groupby(["Mois"]).mean(), 
               labels = {"value": "W/m2", "variable": ""},
               color_discrete_map = {"Vitesse du vent à 100m (m/s)": "red"})

fig2.update_traces(yaxis = "y2", line = dict(width = 2,dash = 'dot'))

subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title = "Mois"
subfig.layout.yaxis2.title = "Vitesse du vent à 100m (m/s)"
subfig.layout.yaxis.title = "FC, %"

subfig.show()

In [ ]:
!pip install install click<8.0,>=5.1
!pip3 install prefect
from prefect import task, Flow
!pip install asyncio



/bin/bash: 8.0,: No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
